In [1]:
import math
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
max_degree = 20  # 多项式的最大阶数
n_train, n_test = 100, 100  # 训练和测试数据集大小
true_w = np.zeros(max_degree)  # 分配大量的空间
true_w[0:4] = np.array([5, 1.2, -3.4, 5.6])

features = np.random.normal(size=(n_train + n_test, 1))
np.random.shuffle(features)
poly_features = np.power(features, np.arange(max_degree).reshape(1, -1))
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1)  # gamma(n)=(n-1)!
# labels的维度:(n_train+n_test,)
labels = np.dot(poly_features, true_w)
labels += np.random.normal(scale=0.1, size=labels.shape)

In [3]:
# NumPy ndarray转换为tensor
true_w, features, poly_features, labels = [torch.tensor(x, dtype=
torch.float32) for x in [true_w, features, poly_features, labels]]

In [4]:
features[:2], poly_features[:2, :], labels[:2]

(tensor([[-1.4449],
         [ 0.0603]]),
 tensor([[ 1.0000e+00, -1.4449e+00,  1.0439e+00, -5.0281e-01,  1.8163e-01,
          -5.2490e-02,  1.2641e-02, -2.6093e-03,  4.7129e-04, -7.5665e-05,
           1.0933e-05, -1.4362e-06,  1.7293e-07, -1.9221e-08,  1.9838e-09,
          -1.9110e-10,  1.7258e-11, -1.4669e-12,  1.1775e-13, -8.9551e-15],
         [ 1.0000e+00,  6.0277e-02,  1.8166e-03,  3.6500e-05,  5.5003e-07,
           6.6308e-09,  6.6614e-11,  5.7361e-13,  4.3219e-15,  2.8946e-17,
           1.7447e-19,  9.5607e-22,  4.8024e-24,  2.2267e-26,  9.5870e-29,
           3.8525e-31,  1.4513e-33,  5.1460e-36,  1.7233e-38,  5.4669e-41]]),
 tensor([-3.0525,  5.0740]))

In [5]:
def evaluate_loss(net, data_iter, loss):  #@save
    """评估给定数据集上模型的损失"""
    metric = d2l.Accumulator(2)  # 损失的总和,样本数量
    for X, y in data_iter:
        out = net(X)
        y = y.reshape(out.shape)
        l = loss(out, y)
        metric.add(l.sum(), l.numel())
    return metric[0] / metric[1]

In [6]:
def train(train_features, test_features, train_labels, test_labels,
          num_epochs=400):
    loss = nn.MSELoss(reduction='none')
    input_shape = train_features.shape[-1]
    # 不设置偏置，因为我们已经在多项式中实现了它
    net = nn.Sequential(nn.Linear(input_shape, 1, bias=False))
    batch_size = min(10, train_labels.shape[0])
    train_iter = d2l.load_array((train_features, train_labels.reshape(-1, 1)),
                                batch_size)
    test_iter = d2l.load_array((test_features, test_labels.reshape(-1, 1)),
                               batch_size, is_train=False)
    trainer = torch.optim.SGD(net.parameters(), lr=0.01)
    animator = d2l.Animator(xlabel='epoch', ylabel='loss', yscale='log',
                            xlim=[1, num_epochs], ylim=[1e-3, 1e2],
                            legend=['train', 'test'])
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (epoch + 1) % 20 == 0:
            animator.add(epoch + 1, (evaluate_loss(net, train_iter, loss),
                                     evaluate_loss(net, test_iter, loss)))
    print('weight:', net[0].weight.data.numpy())

In [7]:
# 从多项式特征中选择前4个维度，即1,x,x^2/2!,x^3/3!
train(poly_features[:n_train, :4], poly_features[n_train:, :4],
      labels[:n_train], labels[n_train:])

<Figure size 640x480 with 0 Axes>

weight: [[ 4.9853034  1.2077818 -3.3774319  5.604991 ]]


In [8]:
# 从多项式特征中选择前2个维度，即1和x
train(poly_features[:n_train, :2], poly_features[n_train:, :2],
      labels[:n_train], labels[n_train:])

<Figure size 640x480 with 0 Axes>

weight: [[2.8897812 4.8723006]]


In [9]:
# 从多项式特征中选取所有维度
train(poly_features[:n_train, :], poly_features[n_train:, :],
      labels[:n_train], labels[n_train:], num_epochs=1500)

<Figure size 640x480 with 0 Axes>

weight: [[ 4.9681125   1.3243725  -3.3117082   5.096156   -0.16184597  1.0432327
   0.29336149 -0.0164045   0.24939655  0.10946755  0.1829925  -0.02115814
  -0.09886038  0.11912026 -0.0514273   0.05942735 -0.15103035 -0.13261144
   0.13380918 -0.21212894]]
